In [5]:
#!pip install --upgrade autogen openai



  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/c7/d8/3e4cf8a5f544bef575d3502fedd81a15e317f591022de940647bdd0cc017/openai-1.54.4-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/packages/2b/30/6a79fd25f36660cec4fb46c5fd0d52375584fdc7a874889b24111cb666af/jiter-0.7.1-cp311-none-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions<5,>=4.11 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/389.6 kB ? eta -:--:--
   --- ------------------------------------ 30.7/389.6 kB ? eta -:--:--
   --------------- ------------------------ 153.6/389.6 kB 2.3 MB/s eta 0:00:01
   ------------------------------- -------- 307.2/389.6 kB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 389.6/389.6 k

In [4]:
import os
import requests
import json
from datetime import datetime, timedelta
from gtts import gTTS
import openai
from openai.types import Completion, CompletionChoice, CompletionUsage

# Configure API keys
news_api_key = os.getenv("news_api_key")
openai_api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_api_key

# Path to the context file
CONTEXT_FILE = "context.json"

# Debugging helper to log messages
def debug_log(message):
    print(f"[DEBUG] {message}")

# Helper functions for context management
def load_context():
    """Load or initialize the context file."""
    if os.path.exists(CONTEXT_FILE):
        with open(CONTEXT_FILE, "r") as f:
            return json.load(f)
    else:
        return {"read_stories": [], "familiar_topics": []}

def save_context(context):
    """Save the updated context file."""
    with open(CONTEXT_FILE, "w") as f:
        json.dump(context, f, indent=4)

# Fetch news articles
def fetch_news(category="general", country="us"):
    debug_log("Fetching news articles...")
    context = load_context()
    read_stories = context["read_stories"]
    cutoff_date = datetime.now() - timedelta(days=3)

    base_url = "https://newsapi.org/v2/top-headlines"
    params = {
        "category": category,
        "country": country,
        "apiKey": news_api_key
    }
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        raise Exception(f"Error fetching news: {response.status_code} - {response.text}")

    articles = response.json().get("articles", [])
    # Filter out articles already read or older than 3 days
    filtered_articles = [
        article for article in articles
        if article.get("url") not in read_stories and
        datetime.strptime(article.get("publishedAt", ""), "%Y-%m-%dT%H:%M:%SZ") >= cutoff_date
    ]
    debug_log(f"Fetched {len(filtered_articles)} new articles.")
    return filtered_articles

# Summarize articles using OpenAI
def summarize_articles(articles):
    debug_log("Summarizing articles...")
    summaries = []
    for article in articles:
        content = f"Title: {article.get('title', '')}\nDescription: {article.get('description', '')}\nContent: {article.get('content', '')}"
        response = openai.ChatCompletion.acreate(  # Use async create if supported
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a summarization assistant for news articles."},
                {"role": "user", "content": content}
            ]
        )
        summaries.append({
            "url": article.get("url"),
            "summary": response.choices[0].message.content  # Correct way to access content
        })
    debug_log(f"Generated {len(summaries)} summaries.")
    return summaries

# Generate audio files from summaries using gTTS
def generate_audio(summaries, time_of_day="morning"):
    debug_log("Generating audio files...")
    audio_files = []
    for idx, summary in enumerate(summaries):
        filename = f"{time_of_day}_summary_{idx + 1}.mp3"
        try:
            tts = gTTS(text=summary["summary"], lang="en")
            tts.save(filename)
            audio_files.append({"url": summary["url"], "file": filename})
        except Exception as e:
            debug_log(f"Error generating audio for summary {idx + 1}: {e}")
    debug_log(f"Generated {len(audio_files)} audio files.")
    return audio_files

# Main workflow
def execute_workflow():
    try:
        # Step 1: Fetch news
        articles = fetch_news(category="general", country="us")
        if not articles:
            debug_log("No new articles available.")
            return

        # Step 2: Summarize articles
        summaries = summarize_articles(articles)

        # Step 3: Convert summaries to audio
        audio_files = generate_audio(summaries, time_of_day="morning")

        # Step 4: Update context
        context = load_context()
        for audio_file in audio_files:
            context["read_stories"].append(audio_file["url"])
        save_context(context)

        # Step 5: Organize and save audio files
        date_str = datetime.now().strftime("%Y-%m-%d")
        directory = os.path.join("daily_news", date_str)
        os.makedirs(directory, exist_ok=True)
        for audio_file in audio_files:
            new_path = os.path.join(directory, os.path.basename(audio_file["file"]))
            os.rename(audio_file["file"], new_path)

        debug_log(f"Daily news audio saved in: {directory}")
    except Exception as e:
        debug_log(f"Error in workflow: {e}")

# Execute the workflow
execute_workflow()


[DEBUG] Fetching news articles...
[DEBUG] Fetched 20 new articles.
[DEBUG] Summarizing articles...
[DEBUG] Error in workflow: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742



In [2]:
# Assumed installed dependencies (uncomment these lines if needed)
# !pip install openai
# !pip install requests
# !pip install pydub
# !pip install autogen

import os
import requests
import json
from datetime import datetime, timedelta
import openai
from pydub import AudioSegment
from autogen import Agent, GroupChat

# Configure API keys
news_api_key = "c6526afc70be458f8e8fdc4f4b443c1c"
openai_api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_api_key

# Path to the context file
CONTEXT_FILE = "context.json"

# Helper functions for context management
def load_context():
    """Load or initialize the context file."""
    if os.path.exists(CONTEXT_FILE):
        with open(CONTEXT_FILE, "r") as f:
            return json.load(f)
    else:
        return {"read_stories": [], "familiar_topics": []}

def save_context(context):
    """Save the updated context file."""
    with open(CONTEXT_FILE, "w") as f:
        json.dump(context, f, indent=4)

# Define individual agents
class NewsFetcherAgent(Agent):
    def __init__(self, name):
        super().__init__(name=name)

    def fetch_news(self, category="general", country="us"):
        """Fetch news articles."""
        context = load_context()
        read_stories = context["read_stories"]
        cutoff_date = datetime.now() - timedelta(days=3)

        base_url = "https://newsapi.org/v2/top-headlines"
        params = {
            "category": category,
            "country": country,
            "apiKey": news_api_key
        }
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            raise Exception(f"Error fetching news: {response.status_code} - {response.text}")

        articles = response.json().get("articles", [])
        # Filter out articles already read or older than 3 days
        return [
            article for article in articles
            if article.get("url") not in read_stories and
            datetime.strptime(article.get("publishedAt", ""), "%Y-%m-%dT%H:%M:%SZ") >= cutoff_date
        ]

class SummarizerAgent(Agent):
    def __init__(self, name):
        super().__init__(name=name)

    def summarize(self, articles):
        """Summarize articles using OpenAI."""
        summaries = []
        for article in articles:
            content = f"Title: {article.get('title', '')}\nDescription: {article.get('description', '')}\nContent: {article.get('content', '')}"
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a summarization assistant for news articles."},
                    {"role": "user", "content": content}
                ]
            )
            summaries.append({
                "url": article.get("url"),
                "summary": response.choices[0].message["content"]
            })
        return summaries

class TextToSpeechAgent(Agent):
    def __init__(self, name):
        super().__init__(name=name)

    def generate_audio(self, summaries, time_of_day="morning"):
        """Convert text summaries to audio files using OpenAI Whisper."""
        audio_files = []
        for idx, summary in enumerate(summaries):
            filename = f"{time_of_day}_summary_{idx + 1}.mp3"
            try:
                # Generate audio file
                response = openai.Audio.create(
                    model="whisper-1",
                    text=summary["summary"]
                )
                with open(filename, "wb") as audio_file:
                    audio_file.write(response.audio)
                audio_files.append({"url": summary["url"], "file": filename})
            except Exception as e:
                raise Exception(f"Error generating audio for summary {idx + 1}: {e}")
        return audio_files

# Instantiate agents
news_fetcher = NewsFetcherAgent(name="NewsFetcher")
summarizer = SummarizerAgent(name="Summarizer")
text_to_speech = TextToSpeechAgent(name="TextToSpeech")

# Define allowed speaker transitions
allowed_speaker_transitions = {
    news_fetcher: [summarizer],
    summarizer: [text_to_speech],
    text_to_speech: []
}

# Initialize GroupChat
group_chat = GroupChat(
    agents=[news_fetcher, summarizer, text_to_speech],
    messages=[],
    allowed_or_disallowed_speaker_transitions=allowed_speaker_transitions,
    speaker_transitions_type="allowed"
)

# Define the workflow
def execute_workflow():
    # Step 1: Fetch news
    articles = news_fetcher.fetch_news(category="general", country="us")
    if not articles:
        return "No new articles available."

    # Step 2: Summarize news
    summaries = summarizer.summarize(articles)

    # Step 3: Convert summaries to audio
    audio_files = text_to_speech.generate_audio(summaries, time_of_day="morning")

    # Step 4: Update context
    context = load_context()
    for audio_file in audio_files:
        context["read_stories"].append(audio_file["url"])
    save_context(context)

    # Step 5: Organize and save audio files
    date_str = datetime.now().strftime("%Y-%m-%d")
    directory = os.path.join("daily_news", date_str)
    os.makedirs(directory, exist_ok=True)
    for audio_file in audio_files:
        new_path = os.path.join(directory, os.path.basename(audio_file["file"]))
        os.rename(audio_file["file"], new_path)

    return f"Daily news audio saved in: {directory}"

# Execute the workflow
result = execute_workflow()
print(result)


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [35]:
# Assumed installed dependencies (uncomment these lines if needed)
# !pip install openai
# !pip install requests
# !pip install pydub
# !pip install autogen

import os
import requests
import json
from datetime import datetime, timedelta
import openai
from pydub import AudioSegment
from autogen import Agent, GroupChat

# Configure API keys
news_api_key = "c6526afc70be458f8e8fdc4f4b443c1c"
openai_api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_api_key

# Path to the context file
CONTEXT_FILE = "context.json"

# Helper functions for context management
def load_context():
    """Load or initialize the context file."""
    if os.path.exists(CONTEXT_FILE):
        with open(CONTEXT_FILE, "r") as f:
            return json.load(f)
    else:
        return {"read_stories": [], "familiar_topics": []}

def save_context(context):
    """Save the updated context file."""
    with open(CONTEXT_FILE, "w") as f:
        json.dump(context, f, indent=4)

# Define individual agents
class NewsFetcherAgent(Agent):
    def __init__(self, name):
        super().__init__(name=name)

    def fetch_news(self, category="general", country="us"):
        """Fetch news articles."""
        context = load_context()
        read_stories = context["read_stories"]
        cutoff_date = datetime.now() - timedelta(days=3)

        base_url = "https://newsapi.org/v2/top-headlines"
        params = {
            "category": category,
            "country": country,
            "apiKey": news_api_key
        }
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            raise Exception(f"Error fetching news: {response.status_code} - {response.text}")

        articles = response.json().get("articles", [])
        # Filter out articles already read or older than 3 days
        return [
            article for article in articles
            if article.get("url") not in read_stories and
            datetime.strptime(article.get("publishedAt", ""), "%Y-%m-%dT%H:%M:%SZ") >= cutoff_date
        ]

class SummarizerAgent(Agent):
    def __init__(self, name):
        super().__init__(name=name)

    def summarize(self, articles):
        """Summarize articles using OpenAI."""
        summaries = []
        for article in articles:
            content = f"Title: {article.get('title', '')}\nDescription: {article.get('description', '')}\nContent: {article.get('content', '')}"
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a summarization assistant for news articles."},
                    {"role": "user", "content": content}
                ]
            )
            summaries.append({
                "url": article.get("url"),
                "summary": response.choices[0].message["content"]
            })
        return summaries

class TextToSpeechAgent(Agent):
    def __init__(self, name):
        super().__init__(name=name)

    def generate_audio(self, summaries, time_of_day="morning"):
        """Convert text summaries to audio files."""
        audio_files = []
        for idx, summary in enumerate(summaries):
            filename = f"{time_of_day}_summary_{idx + 1}.mp3"
            try:
                response = openai.Audio.transcribe(
                    model="whisper-1",
                    file=summary["summary"]
                )
                with open(filename, "wb") as audio_file:
                    audio_file.write(response)
                audio_files.append({"url": summary["url"], "file": filename})
            except Exception as e:
                raise Exception(f"Error generating audio for summary {idx + 1}: {e}")
        return audio_files

# Instantiate agents
news_fetcher = NewsFetcherAgent(name="NewsFetcher")
summarizer = SummarizerAgent(name="Summarizer")
text_to_speech = TextToSpeechAgent(name="TextToSpeech")

# Define allowed speaker transitions
allowed_speaker_transitions = {
    news_fetcher: [summarizer],
    summarizer: [text_to_speech],
    text_to_speech: []
}

# Initialize GroupChat
group_chat = GroupChat(
    agents=[news_fetcher, summarizer, text_to_speech],
    messages=[],
    allowed_or_disallowed_speaker_transitions=allowed_speaker_transitions,
    speaker_transitions_type="allowed"
)

# Define the workflow
def execute_workflow():
    # Step 1: Fetch news
    articles = news_fetcher.fetch_news(category="general", country="us")
    if not articles:
        return "No new articles available."

    # Step 2: Summarize news
    summaries = summarizer.summarize(articles)

    # Step 3: Convert summaries to audio
    audio_files = text_to_speech.generate_audio(summaries, time_of_day="morning")

    # Step 4: Update context
    context = load_context()
    for audio_file in audio_files:
        context["read_stories"].append(audio_file["url"])
    save_context(context)

    # Step 5: Organize and save audio files
    date_str = datetime.now().strftime("%Y-%m-%d")
    directory = os.path.join("daily_news", date_str)
    os.makedirs(directory, exist_ok=True)
    for audio_file in audio_files:
        new_path = os.path.join(directory, os.path.basename(audio_file["file"]))
        os.rename(audio_file["file"], new_path)

    return f"Daily news audio saved in: {directory}"

# Execute the workflow
result = execute_workflow()
print(result)


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
